# Streaming responses

The framework allows to stream responses to the client. Unlike basic LLM calls, streaming responses in an agentic application is less intuitive.

The concept is `Conversation` object represents state (and since every other class is staless by design) and thus we stream updates performed over it. `Workflow` class "listens" for those updates and returns a generator that yields intermediate responses.

## Format

Each response has format `MARKER CONTENT` where `MARKER` is one of `start`, `delta`, `response`, `end` and `CONTENT` is either a string or a JSON object.

- `start` - marks the beginning of a block, which is usually an agent or a team. In this case, `CONTENT` is the name of the agent or the team.
- `delta` - marks a change in the conversation state. `CONTENT` is a JSON object with the following keys:
  - `content` - the text of the response from LLM
  - `function_call` - the function call that was made
  - `tool_calls` - the tool calls that were made

- `response` - marks the final completed response of a block. `CONTENT` is a JSON object with the following
    - `content` - the text of the response from LLM
    - `role` - the role of the agent
    - `name` - the name of the agent
    - `completion_tokens` - the number of tokens in the response
    - `prompt_tokens` - the number of tokens in the prompt
    - `total_tokens` - the total number of tokens

- `end` - marks the end of a block, which is usually an agent or a team. In this case, `CONTENT` is the name of the agent or the team.

## Example output

```text
start: team
start: agent1
delta: {'content': None, 'function_call': None, 'refusal': None, 'tool_calls': [{'id': None, 'function': {'arguments': '{}', 'name': None}, 'type': None}]}
delta: {'content': None, 'function_call': None, 'refusal': None, 'tool_calls': None}
delta: {'content': None, 'function_call': None, 'refusal': None, 'tool_calls': [{'id': 'call_tWwGHxwNbBCx2S7n2ItQsSYF', 'function': {'arguments': '', 'name': 'get_delivery_date'}, 'type': 'function'}]}
delta: {'content': None, 'function_call': None, 'refusal': None, 'tool_calls': [{'id': None, 'function': {'arguments': '{"', 'name': None}, 'type': None}]}
delta: {'content': None, 'function_call': None, 'refusal': None, 'tool_calls': [{'id': None, 'function': {'arguments': 'order', 'name': None}, 'type': None}]}
delta: {'content': None, 'function_call': None, 'refusal': None, 'tool_calls': [{'id': None, 'function': {'arguments': '_id', 'name': None}, 'type': None}]}
delta: {'content': None, 'function_call': None, 'refusal': None, 'tool_calls': [{'id': None, 'function': {'arguments': '":"', 'name': None}, 'type': None}]}
delta: {'content': None, 'function_call': None, 'refusal': None, 'tool_calls': [{'id': None, 'function': {'arguments': '214', 'name': None}, 'type': None}]}
delta: {'content': None, 'function_call': None, 'refusal': None, 'tool_calls': [{'id': None, 'function': {'arguments': '"}', 'name': None}, 'type': None}]}
delta: {'content': None, 'function_call': None, 'refusal': None, 'tool_calls': None}
delta: {'content': '', 'function_call': None, 'refusal': None, 'tool_calls': None}
delta: {'content': 'The', 'function_call': None, 'refusal': None, 'tool_calls': None}
delta: {'content': ' expected', 'function_call': None, 'refusal': None, 'tool_calls': None}
delta: {'content': ' delivery', 'function_call': None, 'refusal': None, 'tool_calls': None}
delta: {'content': ' date', 'function_call': None, 'refusal': None, 'tool_calls': None}
delta: {'content': ' for', 'function_call': None, 'refusal': None, 'tool_calls': None}
delta: {'content': ' your', 'function_call': None, 'refusal': None, 'tool_calls': None}
delta: {'content': ' order', 'function_call': None, 'refusal': None, 'tool_calls': None}
delta: {'content': ' is', 'function_call': None, 'refusal': None, 'tool_calls': None}
delta: {'content': ' January', 'function_call': None, 'refusal': None, 'tool_calls': None}
delta: {'content': ' ', 'function_call': None, 'refusal': None, 'tool_calls': None}
delta: {'content': '1', 'function_call': None, 'refusal': None, 'tool_calls': None}
delta: {'content': ',', 'function_call': None, 'refusal': None, 'tool_calls': None}
delta: {'content': ' ', 'function_call': None, 'refusal': None, 'tool_calls': None}
delta: {'content': '202', 'function_call': None, 'refusal': None, 'tool_calls': None}
delta: {'content': '2', 'function_call': None, 'refusal': None, 'tool_calls': None}
delta: {'content': '.', 'function_call': None, 'refusal': None, 'tool_calls': None}
delta: {'content': None, 'function_call': None, 'refusal': None, 'tool_calls': None}
response: [{'content': 'The expected delivery date for your order is January 1, 2022.', 'role': 'assistant', 'name': 'agent1'}, {'completion_tokens': 44, 'prompt_tokens': 742, 'total_tokens': 786}]
end: agent1
end: teams

```

In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Add the parent directory to sys.path
import sys, os
sys.path.append(os.path.abspath(os.path.join('../vanilla_aiagents')))

from vanilla_aiagents.agent import Agent
from vanilla_aiagents.workflow import Workflow
from vanilla_aiagents.team import Team
from vanilla_aiagents.user import User
from vanilla_aiagents.llm import AzureOpenAILLM
from dotenv import load_dotenv

load_dotenv(override=True)

In [3]:
llm = AzureOpenAILLM({
    "azure_deployment": os.getenv("AZURE_OPENAI_MODEL"),
    "azure_endpoint": os.getenv("AZURE_OPENAI_ENDPOINT"),
    "api_key": os.getenv("AZURE_OPENAI_KEY"),
    "api_version": os.getenv("AZURE_OPENAI_API_VERSION"),
})

import logging

# Set logging to debug
logging.basicConfig(level=logging.INFO)
logging.getLogger("vanilla_aiagents.agent").setLevel(logging.DEBUG)
logging.getLogger("vanilla_aiagents.workflow").setLevel(logging.DEBUG)
logging.getLogger("vanilla_aiagents.conversation").setLevel(logging.DEBUG)
logging.getLogger("vanilla_aiagents.llm").setLevel(logging.DEBUG)

In [ ]:
from typing import Annotated


agent1 = Agent(id="agent1", llm=llm, description="Call this agent for general purpose questions", system_message = """You are an AI assistant
    Your task is to help the user with their questions.
    Always respond with the best answer you can generate.
    If you don't know the answer, respond with "I don't know".
    Always be polite and helpful.
    """)


@agent1.register_tool(description="Get the order ID for a customer's order. Call this whenever you need to know the order ID, for example when a customer asks 'Where is my package'")
def get_order_id() -> Annotated[str, "The customer's order ID."]:
    return "214"

@agent1.register_tool(description="Get the delivery date for a customer's order. Call this whenever you need to know the delivery date, for example when a customer asks 'Where is my package'")
def get_delivery_date(order_id: Annotated[str, "The customer's order ID."]) -> Annotated[str, "The delivery date for the customer's order."]:
    return "2022-01-01"

user = User(id="user", mode="unattended")

flow = Team(id="team", description="", members=[agent1, user], llm=llm, stop_callback=lambda msgs: len(msgs) > 4)



In [ ]:
workflow = Workflow(askable=flow)
stream = workflow.run_stream("Which is expected delivery date for my order?")
for mark, content in stream:
    print(f"{mark}: {content}")